## Projet de MODS206 - Nowcasting and Dynamic Factor Models

### BASTIDE Simon, RISTORCELLI Ulysse, VUJASINOVIC Thomas

Analyse des bases de données de base, en se restreignant aux fréquences mensuelles :

In [7]:
#imports
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import yfinance as yf
import plotly.graph_objects as go
import plotly.io as pio
import datetime as dt
pio.templates.default = "plotly_white"

In [8]:
#imports de capitalisations boursières des grandes entreprises du tourisme
def get_data(assets, start_date, end_date):
    """
    Récupère les données des actions sur Yahoo Finance
    """
    return yf.download(assets, start=start_date, end=end_date)['Adj Close']

assets = ['0H59.IL', 'ALVDM.PA']
N = 5000 # Complétez avec le nombre de jours historiques

start_date = dt.datetime.now() - dt.timedelta(days=N)
end_date = dt.datetime.now()

data = get_data(assets, start_date, end_date)
data
monthly_data = data.resample('M').mean()
monthly_data


[*********************100%%**********************]  2 of 2 completed


,0H59.IL,ALVDM.PA
Date,,
2010-07-31,19.698265,15.172885
2010-08-31,19.460198,15.454809
2010-09-30,20.874527,15.673764
2010-10-31,22.767187,17.284561
2010-11-30,24.577960,18.513290
...,...,...
2023-11-30,30.999091,112.863636
2023-12-31,34.373947,120.447368
2024-01-31,35.514091,124.954545


In [9]:
# Affichage des données superposé avec plotly pour une année 
fig = go.Figure()
for a in assets:
    fig.add_trace(go.Scatter(x=data.index, y=data[a], name=a))
fig.update_layout(title='Evolution des prix des actifs', yaxis_title='Prix en $')
fig.show()

In [11]:
sight_seeing_demand_df=pd.read_csv('databases/famille_SUIVI-DEMANDE-TOURISTIQUE/valeurs_mensuelles.csv', sep=';')
sight_seeing_demand_df=sight_seeing_demand_df.drop(sight_seeing_demand_df[sight_seeing_demand_df['Libellé'] == 'Codes'].index)
sight_seeing_demand_df=sight_seeing_demand_df.drop('Période', axis=1)
sight_seeing_demand_df

overnight_stays_df=pd.read_csv('databases/famille_NUITEE/valeurs_mensuelles.csv', sep=';')
overnight_stays_df=overnight_stays_df.drop(overnight_stays_df[overnight_stays_df['Libellé'] == 'Codes'].index)
overnight_stays_df=overnight_stays_df.drop('Période',axis=1)

regions=[
    "Auvergne-Rhône-Alpes", "Bourgogne-Franche-Comté", "Bretagne",
    "Centre-Val de Loire", "Corse", "Grand Est", "Hauts-de-France",
    "Ile-de-France", "Normandie", "Nouvelle-Aquitaine", "Occitanie",
    "Pays de la Loire", "Provence Alpes Côte d’Azur"
]

overnight_stays_df_t = overnight_stays_df.transpose()
# Rename columns using the first row, cleansing of not useful columns
overnight_stays_df_t.columns = overnight_stays_df_t.iloc[0]
overnight_stays_df_t = overnight_stays_df_t.drop(overnight_stays_df_t.index[:3])

#overnight_stays_df_t = overnight_stays_df_t.iloc[:, list(range(3, 21)) + list(range(64, 80))]
occupation_columns = [col for col in overnight_stays_df_t.columns if "Taux d'occupation dans l'hôtellerie" in col]
overnight_columns = [col for col in overnight_stays_df_t.columns if "Nuitées dans l'hôtellerie - Total -" in col]
arrival_columns= [col for col in overnight_stays_df_t.columns if "Arrivées dans l'hôtellerie - Total -" in col]

selected_occupation_columns = [col for col in occupation_columns if any(region in col for region in regions)]
selected_overnight_columns = [col for col in overnight_columns if any(region in col for region in regions)]
selected_arrival_columns = [col for col in arrival_columns if any(region in col for region in regions)]

selected_columns = selected_arrival_columns+selected_occupation_columns+selected_overnight_columns
overnight_stays_df_t =overnight_stays_df_t[selected_columns]
#overnight_stays_df_t["Taux"]
overnight_stays_df_t

Libellé,Arrivées dans l'hôtellerie - Total - Centre-Val de Loire - Série arrêtée,Arrivées dans l'hôtellerie - Total - Bourgogne-Franche-Comté - Série arrêtée,Arrivées dans l'hôtellerie - Total - Normandie - Série arrêtée,Arrivées dans l'hôtellerie - Total - Hauts-de-France - Série arrêtée,Arrivées dans l'hôtellerie - Total - Grand Est - Série arrêtée,Arrivées dans l'hôtellerie - Total - Pays de la Loire - Série arrêtée,Arrivées dans l'hôtellerie - Total - Bretagne - Série arrêtée,Arrivées dans l'hôtellerie - Total - Nouvelle-Aquitaine - Série arrêtée,Arrivées dans l'hôtellerie - Total - Occitanie - Série arrêtée,Arrivées dans l'hôtellerie - Total - Auvergne-Rhône-Alpes - Série arrêtée,...,Nuitées dans l'hôtellerie - Total - Normandie,Nuitées dans l'hôtellerie - Total - Auvergne-Rhône-Alpes,Nuitées dans l'hôtellerie - Total - Grand Est,Nuitées dans l'hôtellerie - Total - Nouvelle-Aquitaine,Nuitées dans l'hôtellerie - Total - Occitanie,Nuitées dans l'hôtellerie - Total - Corse-du-Sud,Nuitées dans l'hôtellerie - Total - Bourgogne-Franche-Comté,Nuitées dans l'hôtellerie - Total - Haute-Corse,Nuitées dans l'hôtellerie - Total - Centre-Val de Loire,Nuitées dans l'hôtellerie - Total - Hauts-de-France
2010-01,166.87,241.91,210.05,308.91,389.87,207.7,170.45,379.49,389.47,793.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-02,189.11,308.87,252.95,338.58,445.58,218.23,203.79,457.31,461.73,918.92,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-03,249.37,333.78,318.79,403.19,522.0,287.74,264.82,567.07,542.73,971.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-04,315.19,414.1,430.0,476.24,627.02,312.64,344.17,730.62,753.26,932.56,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-05,385.39,474.67,478.52,517.69,731.28,347.41,398.96,793.95,898.14,965.39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,838,2053,1419,1654,1691,274.56,712,206.43,613,886
2023-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,674,1650,1238,1369,1362,141.22,581,87.56,511,805
2023-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,470,1330,1005,913,838,26.35,427,23.84,369,662
2023-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,421,1899,1357,891,834,20.76,427,19.56,353,670
